<a href="https://colab.research.google.com/github/Jade-P/MLGroupProject/blob/Angry--Arandi/Live_Feed_Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Capture

This code will only work on jupyter notebook because google colab does not have the capacity to run a live feed and apply an algorithm. 

**Download modelc.h5 and haarcascade_frontalface_default.xml locally then add path to each file when prompted.**

In [ ]:
#importing necessary packages
import cv2
import numpy as np



#Use the following data if you have the model in the same file
#from tensorflow.keras.utils import img_to_array

#Use this if you are loading the model into notebook 
from tensorflow.keras.preprocessing.image import img_to_array


#Naming Window
cv2.namedWindow("Face Detect")

#Load Model function
from tensorflow.keras.models import load_model



#apply path of modelc FOUND IN GITHUB REPOSITORY, DOWNLOAD LOCALLY
path = "/modelc.h5"
model = load_model(path)

In [ ]:
#This will help ensure that you have the corect model 
model.summary()

In [ ]:
#intializing Classifier 

#apply path of haarcascade_frontalface_default.xml FOUND IN GITHUB REPOSITORY, DOWNLOAD LOCALLY
path_ ='/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(path_)

In [ ]:
#Labels for the square to output
emotion_labels = ['Angry', 'Fear', 'Happy', 'Neutral', 'Sad', 'Suprise']

#This is initializing the video capture
# '0' is getting the videocapture from your local machine 
cap = cv2.VideoCapture(0)

while(True):
    #this is getting each frame
    ret,frame = cap.read()
    labels = []
    #converting to gray scale 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #finding the faces from the classifier 
    faces =face_cascade.detectMultiScale(gray, 1.1, 4)
    #the following will apply to a found face 
    for (x,y,w,h) in faces:
        #preprocessing data
        color = (0, 255, 0)# this is BGR
        stroke = 2 #how thick the line is 
        cv2.rectangle(frame, (x,y),(x+w, y+h), color , stroke) #draws the rectangle
        roi_gray = gray[y:y+h, x:x+w] #getting cordinates of faces
        roi_gray = cv2.resize(roi_gray, (48,48), interpolation = cv2.INTER_AREA) #converting becasue the model needs a 48x48 image
        
        #recognize --- add Deep Learning model 
        if np.sum([roi_gray])!= 0: #insuring that a face is there
            roi = roi_gray.astype('float')/255.0  
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            
            #adding model
            classifier = model
            prediction = classifier.predict(roi)[0] #applying model
            label = emotion_labels[prediction.argmax()] #labeling model 
            label_pos = (x,y-10) #where the label will be 
            cv2.putText(frame, label, label_pos, cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),stroke) #outputting labels to model
        else:
            #If no face is there
            cv2.putText(frame, "Can't Detect Emotion", (30,80), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),stroke)
    #Shows the found lables and frame 
    cv2.imshow('Emotion Detector', frame)
    #displays the resulting frame, and when user presses 'q' the feed will end
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#Ending capture
cap.release
cv2.destroyAllWindows()
exit()